## Initializing Dataset

In [ ]:
# Mounting drive and importing dataset from google drive

from google.colab import drive

drive.mount("/content/gdrive")

In [ ]:
!unzip /content/gdrive/MyDrive/Copy\ of\ dataset.zip

In [1]:
import numpy as np
from PIL import Image
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
import pandas as pd
import torchvision
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
import os
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
import random

In [2]:
numpy_no = []
numpy_sphere = []
numpy_vort = []
numpy_no_val = []
numpy_sphere_val = []
numpy_vort_val = []

for np_name in glob.glob('/content/dataset/train/no/*.np[yz]'):
    numpy_no.append(np.squeeze(np.load(np_name)))


for np_name in glob.glob('/content/dataset/train/sphere/*.np[yz]'):
    numpy_sphere.append(np.squeeze(np.load(np_name)))


for np_name in glob.glob('/content/dataset/train/vort/*.np[yz]'):
    numpy_vort.append(np.squeeze(np.load(np_name)))


for np_name in glob.glob('/content/dataset/val/no/*.np[yz]'):
    numpy_vort.append(np.squeeze(np.load(np_name)))


for np_name in glob.glob('/content/dataset/val/sphere/*.np[yz]'):
    numpy_vort.append(np.squeeze(np.load(np_name)))


for np_name in glob.glob('/content/dataset/val/vort/*.np[yz]'):
    numpy_vort.append(np.squeeze(np.load(np_name)))

In [3]:
nox_train, nox_test, noy_train, noy_test = train_test_split(numpy_no, np.zeros(len(numpy_no)), test_size=0.1)
spherex_train, spherex_test, spherey_train, spherey_test = train_test_split(numpy_sphere, np.ones(len(numpy_sphere)), test_size=0.1)
vortx_train, vortx_test, vorty_train, vorty_test = train_test_split(numpy_vort, np.full(len(numpy_vort), 2), test_size=0.1)

In [4]:
X_train = nox_train + spherex_train + vortx_train
X_test = nox_test + spherex_test + vortx_test
Y_train = np.concatenate((noy_train, spherey_train, vorty_train), axis=None)
Y_test = np.concatenate((noy_test, spherey_test, vorty_test), axis=None)
X_val = numpy_no_val + numpy_sphere_val + numpy_vort_val
Y_val = np.concatenate((np.zeros(len(numpy_no_val)), np.ones(len(numpy_sphere_val)), np.full(len(numpy_vort_val), 2)), axis=None)

In [5]:
mapIndexPosition = list(zip(X_train, Y_train))
random.shuffle(mapIndexPosition)
# make list separate
X_train, Y_train = zip(*mapIndexPosition)

In [6]:
X_train = list(X_train)
Y_train = np.array(list(Y_train))

## DataLoader

In [7]:
class ToNumpy(object):
    def __call__(self, sample):
        return np.array(sample)

def fix_compose_transform(transform):
        if isinstance(transform.transforms[-1], torchvision.transforms.ToTensor):
            transform = torchvision.transforms.Compose([
                *transform.transforms[:-1],
                ToNumpy(),
                torchvision.transforms.ToTensor()
            ])
        return transform


In [8]:
class CustomDataset(Dataset):
    def __init__(self, X_train, Y_train, X_test, Y_test, X_val, Y_val, phase):
        self.phase = phase
        self.X_train = X_train
        self.Y_train = Y_train
        self.X_test = X_test
        self.Y_test = torch.from_numpy(Y_test).type(torch.LongTensor)
        self.tensor_Y_train = torch.from_numpy(Y_train).type(torch.LongTensor)
        self.X_val = X_val
        self.Y_val = torch.from_numpy(Y_val).type(torch.LongTensor)

    def __getitem__(self, idx):
        if self.phase == 'train':

            trainsample = self.X_train[idx]
            im = Image.fromarray(trainsample)

            transform = transforms.Compose([
                                            # transforms.RandomRotation(45),
                                            # transforms.RandomHorizontalFlip(),
                                            # transforms.RandomVerticalFlip(),
                                            # transforms.Resize((150, 150)),
                                            transforms.ToTensor()
                                            ])
            
            fix_transform = fix_compose_transform(transform)
            data = fix_transform(im)
            label = self.tensor_Y_train[idx]
            return data, label

        elif self.phase == 'test':

            testsample = self.X_test[idx]
            im = Image.fromarray(testsample)

            transform = transforms.Compose([
                                            # transforms.Resize((150, 150)),
                                            transforms.ToTensor()
                                            ])
            
            fix_transform = fix_compose_transform(transform)
            data = fix_transform(im)
            label = self.Y_test[idx]
            return data, label

        elif self.phase == 'val':
            testsample = self.X_test[idx]
            im = Image.fromarray(testsample)

            transform = transforms.Compose([
                                            # transforms.Resize((150, 150)),
                                            transforms.ToTensor()
                                            ])
            
            fix_transform = fix_compose_transform(transform)
            data = fix_transform(im)
            label = self.Y_test[idx]
            return data, label
            
            
    def __len__(self):
        if self.phase == "train":
          return len(X_train)
        else:
          return len(X_test)

In [38]:
training_data = CustomDataset(X_train, Y_train, X_test, Y_test, X_val, Y_val, 'train')
testing_data = CustomDataset(X_train, Y_train, X_test, Y_test, X_val, Y_val, 'test')
validation_data = CustomDataset(X_train, Y_train, X_test, Y_test, X_val, Y_val, 'val')

In [39]:
train_loader = DataLoader(training_data, batch_size=100, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=100, shuffle=False)
val_loader = DataLoader(validation_data, batch_size=100, shuffle=False)

In [11]:
del numpy_no
del numpy_sphere
del numpy_vort
del numpy_no_val
del numpy_sphere_val
del numpy_vort_val

## Model

In [12]:
import torch.nn as nn
import torch.nn.functional as F

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                    stride=stride, padding=1, bias=False)



class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out



class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(1, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AdaptiveAvgPool2d(8)
        self.fc1 = nn.Linear(8*8*64, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 250)
        self.fc4 = nn.Linear(250, 100)
        self.fc5 = nn.Linear(100, 3)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(-1, 8*8*64)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        out = F.relu(self.fc4(out))
        out = self.fc5(out)
        return out


In [13]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:
batch_size = 10
net_args = {
    "block": ResidualBlock,
    "layers": [2, 2, 2, 2]
}
model = ResNet(**net_args)

model.to(device)
print(device)

cuda:0


In [14]:
def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)


In [15]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

In [ ]:
epochs = 5
min_valid_loss = np.inf
 
for e in range(epochs):
    train_loss = 0.0
    for data, labels in train_loader:
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
         
        # Clear the gradients
        optimizer.zero_grad()
        # Forward Pass
        target = model(data)
        # Find the Loss
        loss = criterion(target,labels)
        # Calculate gradients
        loss.backward()
        # Update Weights
        optimizer.step()
        # Calculate Loss
        train_loss += loss.item()
     
    valid_loss = 0.0
    model.eval()     # Optional when not using Model Specific layer
    for data, labels in val_loader:
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
         
        # Forward Pass
        target = model(data)
        # Find the Loss
        loss = criterion(target,labels)
        # Calculate Loss
        valid_loss += loss.item()
 
    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(val_loader)} \t\t Accuracy: {testAccuracy()}')
     
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
         
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')

## Loss and Accuracy Plots - ROC Curve


In [18]:
torch.cuda.empty_cache()

In [16]:
test_loader = DataLoader(testing_data, batch_size=1, shuffle=False)

In [32]:
model.eval()
accuracy = 0.0
total = 0.0

y_pred = []

with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        # run the model on the test set to predict labels
        outputs = model(images)
        # the label with the highest energy will be our prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        accuracy += (predicted == labels).sum().item()
        y_pred.append(predicted.cpu().numpy()[0])

# compute the accuracy over all test images
accuracy = (100 * accuracy / total)



In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
fpr, tpr, threshold = metrics.roc_curve(Y_test, y_pred, pos_label=1)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()